[View in Colaboratory](https://colab.research.google.com/github/mitreilab/autonomousfuture/blob/master/reilAutomation.ipynb)

## Import OES Data

In [0]:
import pandas
oes = pandas.read_csv("https://raw.githubusercontent.com/mitreilab/autonomousfuture/master/oes/realestate_oes_nat4d_2007-2016.csv", low_memory = False)


## Mount Google Drive

In [21]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive"

TIMEOUT: ignored

## Import ONet Data

In [20]:
from google.colab import drive
drive.mount('/content/drive')

import sqlite3
# onetConnectioGithub = sqlite3.connect("https://raw.githubusercontent.com/mitreilab/autonomousfuture/master/onet/onet.sqlite")
onetConnection = sqlite3.connect('/content/drive/My Drive/onet.sqlite')
cur = onetConnection.cursor()
query = "SELECT * FROM occupation_data";
# occdata = pandas.read_sql_query(query, onetConnection)
# onetConnection.execute(query)

KeyboardInterrupt: ignored